Imports

In [90]:
import numpy as np
import random
from pysat.solvers import Solver
from collections import deque
from scipy.ndimage import label



Task 1: Generate puzzles

1.1 define puzzle representations

using a 2D array , matrix grid where it is NxN grid size. 

(A-Z) is for the region labels
( * ) is for when a star is present in the grid


In [91]:
#example with 5x5 grid 1 star per row/column/region
puzzle = [
 ['A', 'B', 'C', 'C', 'C'],
 ['A', 'B', 'C', 'C', 'D'],
 ['A', 'A', 'C', 'D', 'D'],
 ['E', 'C', 'C', 'D', 'D'],
 ['E', 'E', 'E', 'D', 'D']
 ]

solution = [
 ['0', '1', '0', '0', '0'],
 ['0', '0', '0', '1', '0'],
 ['1', '0', '0', '0', '0'],
 ['0', '0', '0', '0', '1'],
 ['0', '0', '1', '0', '0']
]
# example from puzzle-star-battle.com solved by myself

Similar for 10x10 grid with 2 stars per row/column/region, and even 14x14 grids with 3 stars per row/column/region

In [92]:
print("example with labels")

example with labels


1.2 Constraint representations

each row contains only k stars 
each column contains only k stars 
each region contains only k stars
no two stars can be adjacent

pre compute constraint maps

find values about puzzle

In [93]:
def get_star_limit(grid_size):
    if grid_size == 5:
        return 1
    elif grid_size == 6:
        return 1
    elif grid_size == 8:
        return 1
    elif grid_size == 10:
        return 2
    elif grid_size == 14:
        return 3
    else:
        raise ValueError(f"Grid size {grid_size} not supported")

Precompute and validate puzzles

In [94]:
def validate_solution(puzzle,solution):
    #get number of stars for grid size of puzzle
    grid_size =len(puzzle)
    star_lim = get_star_limit(grid_size)

    unique_regions = set(cell for row in puzzle for cell in row)

    # number of regions matches the grid size
    if len(unique_regions) != grid_size:
        return False

    #track star counts in row, column, region
    row_counts = np.zeros(len(puzzle), dtype = int)
    col_counts = np.zeros(len(puzzle[0]),dtype=int)
    region_counts = {region: 0 for region in set([item for sublist in puzzle for item in sublist])}

    # check puzzle and solution dimentions match
    if len(puzzle) != len(solution) or len(puzzle[0]) != len(solution):
        return False
    
    for r in range(grid_size):
        for c in range(grid_size):
            if solution[r][c] == '1':
                row_counts[r]+=1
                col_counts[c]+=1
                region = puzzle[r][c]

                #count stars in region
                if region not in region_counts:
                    region_counts[region]=0
                region_counts[region]+=1
    
    # check each row,colum,region has exactly n stars 
    if not (all(count == star_lim for count in row_counts) and
            all(count == star_lim for count in col_counts) and
            all(count == star_lim for count in region_counts.values())):
        return False
    
    #ensure stars dont touch diagnoally
    for r in range(grid_size):
        for c in range(grid_size):
            if solution[r][c] == '1':
                # check adjacent cells for stars
                for dr in [-1, 0, 1]:
                    for dc in [-1, 0, 1]:
                        if dr == 0 and dc == 0:
                            continue
                        nr, nc = r + dr, c + dc
                        if 0 <= nr < len(puzzle) and 0 <= nc < len(puzzle[0]):
                            if solution[nr][nc] == '1':
                                return False
                            
    return True

In [95]:
#checking this works
valid = validate_solution(puzzle,solution)
print("solution?", valid)

solution? True


Some example puzzles to test 

In [96]:
#example puzzles 
puzzle_5x5 = [
    ['A', 'B', 'C', 'D', 'E'],
    ['A', 'B', 'C', 'D', 'F'],
    ['A', 'B', 'C', 'G', 'F'],
    ['A', 'H', 'C', 'G', 'F'],
    ['I', 'H', 'C', 'G', 'F']
]

solution_5x5 = [
    ['0', '0', '0', '1', '0'],
    ['0', '0', '1', '0', '0'],
    ['0', '0', '0', '0', '1'],
    ['1', '0', '0', '0', '0'],
    ['0', '1', '0', '0', '0']
]

puzzle_10x10 = [
    ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']
]

solution_10x10 = [
    ['0', '1', '0', '0', '1', '0', '0', '0', '0', '0'],
    ['1', '0', '0', '0', '0', '1', '0', '0', '0', '0'],
    ['0', '0', '1', '0', '0', '0', '1', '0', '0', '0'],
    ['0', '1', '0', '0', '0', '0', '0', '1', '0', '0'],
    ['1', '0', '0', '1', '0', '0', '0', '0', '0', '0'],
    ['0', '0', '0', '0', '1', '0', '1', '0', '0', '0'],
    ['0', '0', '1', '0', '0', '0', '0', '1', '0', '0'],
    ['1', '0', '0', '1', '0', '0', '0', '0', '0', '0'],
    ['0', '1', '0', '0', '0', '1', '0', '0', '0', '0'],
    ['0', '0', '1', '0', '0', '0', '0', '0', '1', '0']
]

puzzle_14x14 = [
    ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N'],
    ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N'],
    ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N'],
    ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N'],
    ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N'],
    ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N'],
    ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N'],
    ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N'],
    ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N'],
    ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N'],
    ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N'],
    ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N'],
    ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N'],
    ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N'],
]

solution_14x14 = [
    ['0', '0', '1', '0', '0', '0', '1', '0', '0', '1', '0', '0', '0', '0'],
    ['0', '1', '0', '0', '0', '1', '0', '0', '1', '0', '0', '0', '1', '0'],
    ['1', '0', '0', '1', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0'],
    ['0', '0', '0', '0', '0', '1', '0', '1', '0', '1', '0', '0', '0', '0'],
    ['1', '0', '1', '0', '0', '0', '0', '1', '0', '0', '1', '0', '0', '0'],
    ['0', '1', '0', '0', '1', '0', '0', '0', '0', '1', '1', '0', '0', '0'],
    ['0', '0', '1', '0', '0', '0', '0', '1', '0', '1', '0', '0', '0', '0'],
    ['0', '1', '0', '0', '0', '0', '1', '0', '0', '0', '0', '1', '0', '0'],
    ['1', '0', '0', '0', '1', '0', '0', '0', '0', '0', '1', '0', '1', '0'],
    ['0', '1', '0', '1', '0', '0', '0', '0', '1', '0', '0', '1', '0', '0'],
    ['1', '0', '0', '0', '0', '0', '1', '1', '0', '0', '1', '0', '0', '0'],
    ['0', '0', '0', '1', '1', '0', '0', '0', '1', '0', '1', '0', '0', '0'],
    ['0', '0', '1', '0', '0', '0', '0', '1', '0', '0', '1', '0', '0', '0'],
    ['1', '0', '0', '0', '1', '1', '0', '0', '0', '0', '0', '0', '0', '0']
]


In [97]:
print(validate_solution(puzzle_5x5,solution_5x5),
      validate_solution(puzzle_10x10, solution_10x10),
      validate_solution(puzzle_14x14, solution_14x14))

False False False


1.3 Basic Generation Algorithm using sat solver

In [98]:
def validate_stars(grid, star_lim):
    size = len(grid)
    
    # Check each column has exact num stars
    if not all(np.sum(grid[:, col]) == star_lim for col in range(size)):
        return False

    # Check no adjacent stars
    for r in range(size):
        for c in range(size):
            if grid[r, c] == 1:
                for dr in [-1, 0, 1]:
                    for dc in [-1, 0, 1]:
                        if (dr == 0 and dc == 0) or not (0 <= r + dr < size and 0 <= c + dc < size):
                            continue
                        if grid[r + dr, c + dc] == 1:
                            return False
    return True

In [99]:
def adjacent_star(grid, r, c):
    size = len(grid)
    for dr in [-1, 0, 1]:
        for dc in [-1, 0, 1]:
            if dr == 0 and dc == 0:
                continue
            nr, nc = r + dr, c + dc
            if 0 <= nr < size and 0 <= nc < size and grid[nr, nc] == 1:
                return True
    return False

In [100]:
#start
def generate_star_solution(size):
    star_lim = get_star_limit(size)
    grid = np.zeros((size, size), dtype=int)

    # available positions per column to ensure exact star limits
    column_counts = [0] * size
    
    for row in range(size):
        valid_positions = list(range(size))
        random.shuffle(valid_positions)  
        stars_placed = 0

        for col in valid_positions:
            # Check column limit and adjacency
            if column_counts[col] < star_lim and not adjacent_star(grid, row, col):
                grid[row, col] = 1
                column_counts[col] += 1
                stars_placed += 1
                if stars_placed == star_lim:
                    break

    # Ensure final validation
    if not validate_stars(grid, star_lim):
        return generate_star_solution(size)  # Retry in rare failure cases

    return grid

In [110]:
# check first secton
solution1 = generate_star_solution(10)

print(solution1)


[[1 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 1 0 1 0 0 0]
 [0 0 1 0 0 0 0 0 0 1]
 [0 0 0 0 0 1 0 1 0 0]
 [1 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 1 0]
 [0 1 0 0 0 0 1 0 0 0]
 [0 0 0 1 0 0 0 0 1 0]
 [0 1 0 0 0 1 0 0 0 0]
 [0 0 0 1 0 0 0 1 0 0]]


regions

In [104]:
def generate_regions(solution):
    size = len(solution)
    regions = np.full((size, size), -1, dtype=int)  # -1 means unassigned
    region_id = 0
    min_region_size = 2  # Minimum number of cells per region
    max_region_size = size // 1.5  # Prevent overly large regions
    
    # Step 1: Select random start points for regions
    possible_cells = [(r, c) for r in range(size) for c in range(size) if solution[r, c] == 0]
    random.shuffle(possible_cells)

    while possible_cells:
        r, c = possible_cells.pop()
        if regions[r, c] != -1:
            continue  # Skip already assigned cells
        
        # Step 2: Grow a new region using BFS/DFS
        stack = [(r, c)]
        current_region = []
        
        while stack and len(current_region) < max_region_size:
            x, y = stack.pop()
            if regions[x, y] != -1 or solution[x, y] == 1:  # Skip stars & assigned cells
                continue
            
            regions[x, y] = region_id
            current_region.append((x, y))
            
            # Shuffle for more randomness
            neighbors = [(x+dx, y+dy) for dx, dy in [(-1,0), (1,0), (0,-1), (0,1)]]
            random.shuffle(neighbors)

            for nx, ny in neighbors:
                if 0 <= nx < size and 0 <= ny < size and regions[nx, ny] == -1:
                    stack.append((nx, ny))
        
        # Step 3: Check if the region is too small
        if len(current_region) < min_region_size:
            for x, y in current_region:
                regions[x, y] = -1  # Reset
        else:
            region_id += 1  # Only increment if region was valid
    
    # Step 4: Merge any leftover unassigned cells
    for r in range(size):
        for c in range(size):
            if regions[r, c] == -1:  # Find unassigned cells
                for dx, dy in [(-1,0), (1,0), (0,-1), (0,1)]:
                    nr, nc = r+dx, c+dy
                    if 0 <= nr < size and regions[nr, nc] != -1:
                        regions[r, c] = regions[nr, nc]  # Merge to nearest region
                        break

    # Convert numeric regions to lettered format
    unique_regions = np.unique(regions)
    region_map = {val: chr(65 + i) for i, val in enumerate(unique_regions)}
    lettered_regions = np.array([[region_map[val] for val in row] for row in regions])

    return lettered_regions

# # Example usage
# size = 10
# solution = np.random.choice([0, 1], size=(size, size), p=[0.8, 0.2])  # Simulated star solution
# regions = generate_regions(solution)

# print("\nStar Solution Grid:")
# print(solution)
# print("\nGenerated Regions:")
# print(regions)


In [111]:
print(solution1)
solution1 = np.array(solution1,dtype=int)
print(solution1)

[[1 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 1 0 1 0 0 0]
 [0 0 1 0 0 0 0 0 0 1]
 [0 0 0 0 0 1 0 1 0 0]
 [1 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 1 0]
 [0 1 0 0 0 0 1 0 0 0]
 [0 0 0 1 0 0 0 0 1 0]
 [0 1 0 0 0 1 0 0 0 0]
 [0 0 0 1 0 0 0 1 0 0]]
[[1 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 1 0 1 0 0 0]
 [0 0 1 0 0 0 0 0 0 1]
 [0 0 0 0 0 1 0 1 0 0]
 [1 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 1 0]
 [0 1 0 0 0 0 1 0 0 0]
 [0 0 0 1 0 0 0 0 1 0]
 [0 1 0 0 0 1 0 0 0 0]
 [0 0 0 1 0 0 0 1 0 0]]


In [112]:
puzzle1 = generate_regions(solution1)
print(puzzle1)
print(solution1)

[['E' 'E' 'E' 'C' 'C' 'C' 'J' 'J' 'J' 'J']
 ['E' 'E' 'E' 'C' 'C' 'C' 'J' 'J' 'J' 'J']
 ['E' 'E' 'E' 'M' 'C' 'C' 'A' 'A' 'J' 'J']
 ['F' 'F' 'M' 'M' 'M' 'C' 'A' 'A' 'D' 'D']
 ['F' 'F' 'M' 'H' 'M' 'M' 'A' 'A' 'A' 'D']
 ['F' 'F' 'B' 'H' 'M' 'H' 'L' 'L' 'A' 'D']
 ['F' 'F' 'B' 'H' 'H' 'H' 'L' 'L' 'D' 'D']
 ['I' 'I' 'B' 'H' 'K' 'K' 'L' 'L' 'D' 'G']
 ['I' 'I' 'B' 'K' 'K' 'K' 'L' 'G' 'G' 'G']
 ['I' 'B' 'B' 'K' 'K' 'K' 'L' 'G' 'G' 'G']]
[[1 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 1 0 1 0 0 0]
 [0 0 1 0 0 0 0 0 0 1]
 [0 0 0 0 0 1 0 1 0 0]
 [1 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 1 0]
 [0 1 0 0 0 0 1 0 0 0]
 [0 0 0 1 0 0 0 0 1 0]
 [0 1 0 0 0 1 0 0 0 0]
 [0 0 0 1 0 0 0 1 0 0]]


In [113]:
print(f"the solution is {validate_solution(puzzle1, solution1)}")

the solution is False


1.3 Data structure choices

In [108]:
print("examples class")

examples class


1.4 Storing and compiling puzzles

1.4 SAT solve to validate uniqueness of puzzles

1.6 Generate initial test set of puzzles

In [109]:
print("test set of puzzles")

test set of puzzles
